## 算法稳定性检测

In [1]:
import import_ipynb
from at2vec import ctx, sp2id
import logging
from datetime import datetime
# 配置

if __name__ == '__main__':
    
    # 训练集参数
    ctx.min_x = 3417664.5
    ctx.max_x = 4080080.0
    ctx.min_y = 4722390.0
    ctx.max_y = 5445893.5
    ctx.min_ts = 0
    ctx.max_ts = 50
    
    ctx.num_ts_grids = (ctx.max_ts - ctx.min_ts) // ctx.ts_gap + 1
    ctx.x_gap, ctx.y_gap = ((ctx.max_x - ctx.min_x) / ctx.num_x_grids,
                            (ctx.max_y - ctx.min_y) / ctx.num_y_grids)
    ctx.num_sp_grids = sp2id(ctx.max_x, ctx.max_y,
                             ctx.min_x, ctx.min_y,
                             ctx.max_x, ctx.max_y,
                             ctx.x_gap, ctx.y_gap)
    
    ctx.test_tr_path = 'data/brinkhoff-stability'
    ctx.keywords_path = 'data/amap-labels.txt'
    ctx.logging_path = 'data/at2vec.log'
    ctx.test_batch_size = 1024
    
    logging.basicConfig(filename=ctx.logging_path, format='%(message)s', level=logging.INFO)
    logging.info(str(datetime.now()))
    logging.info('running test.ipynb...')
   
ctx.__dict__

{'min_x': 3417664.5,
 'max_x': 4080080.0,
 'min_y': 4722390.0,
 'max_y': 5445893.5,
 'min_ts': 0,
 'max_ts': 50,
 'sampled_tr_path': 'data/geolife-train-ps-80',
 'complete_tr_path': 'data/geolife-train-ps-100',
 'num_x_grids': 200,
 'num_y_grids': 200,
 'ts_gap': 1,
 'sampled_tr_len': 40,
 'complete_tr_len': 50,
 'ts_rep_dict_path': 'data/ts_rep_dict.pkl',
 'sp_rep_dict_path': 'data/sp_rep_dict.pkl',
 'ts_pretrain_model_path': 'data/ts_pretrain_model.pt',
 'sp_pretrain_model_path': 'data/sp_pretrain_model.pt',
 'sm_pretrain_model_path': 'data/semantic2vec.model',
 'bare_dataset_path': 'data/bare_dataset.pt',
 'dataset_path': 'data/dataset.pt',
 'at2vec_model_path': 'data/at2vec_model.pt',
 'at2vec_rep_path': 'data/at2vec_rep_path.pt',
 'sp_len': 100,
 'ts_len': 100,
 'sm_len': 100,
 'pt_len': 300,
 'hidden_len': 256,
 'k': 10,
 'batch_size': 8,
 'device': device(type='cuda'),
 'alpha': 1,
 'beta': 1,
 'gamma': 1,
 'num_ts_grids': 51,
 'x_gap': 3312.0775,
 'y_gap': 3617.5175,
 'num_sp_g

In [2]:
import torch
from tqdm.notebook import trange

if __name__ == '__main__':
    torch.set_grad_enabled(False)
    
    
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, bare_dataset, raw2tr):
        self.bare_dataset = bare_dataset
        self.vectors = []
        for i in trange(len(self.bare_dataset)):
            _, raw = self.bare_dataset[i]
            self.vectors.append(raw2tr(raw))
        
    def __len__(self):
        return len(self.bare_dataset)
    
    def __getitem__(self, index):
        """
        Returns:
            (index, tr)
        """
        tr = self.vectors[index]
        return index, tr

In [3]:
import torch
import heapq


class Evaluator:
    """
    用于检测单个数据集、单个噪音，并查询前k个结果。
    """
    def __init__(self, model, noise, k):
        self.model = model
        self.noise = noise
        self.k = k
        self.chosen_tr = None
        self.queue = []  # list[distance, index] 大根堆
    
    def set_chosen_tr_vec(self, tr_vec):
        """
        设置基准轨迹的向量表示。
        
        Args:
            tr: 基准轨迹的向量最终表示
        """
        self.chosen_tr_vec = tr_vec.to(ctx.device)
    
    def evaluate(self, index: int, raw, tr):
        """
        先对轨迹加噪音，然后保存并返回离基准轨迹的距离。
        
        Args:
            index: 轨迹编号
            raw: pd.DataFrame 轨迹的原始表示
            tr: torch.Tensor  轨迹的向量表示
        Returns:
            添加噪音后的轨迹离基准轨迹的距离
        """
        tr = tr.to(ctx.device)
        d = torch.dist(self.model.get_rep_vector(
            self.noise.apply(raw, tr).to(ctx.device)), 
            self.chosen_tr_vec).item()
        heapq.heappush(self.queue, (-d, index))
        if len(self.queue) > self.k:
            heapq.heappop(self.queue)
        return d
    
    def evaluate_batch(self, indexes, raw, tr):
        assert len(raw) == tr.shape[0]
        # (batch_size, hidden_len)
        applied = self.model.get_rep_vector(self.noise.apply_batch(raw, tr))
        dists = torch.cdist(applied, self.chosen_tr_vec.unsqueeze(0)).squeeze()
        for i in range(dists.shape[0]):
            heapq.heappush(self.queue, (-dists[i].item(), indexes[i].item()))
            if len(self.queue) > self.k:
                heapq.heappop(self.queue)
                
    def evaluate_batch_applied(self, indexes, applied):
        dists = torch.cdist(applied, self.chosen_tr_vec.unsqueeze(0)).squeeze()
        for i in range(dists.shape[0]):
            pair = (-dists[i].item(), indexes[i].item())
            heapq.heappush(self.queue, pair)
            if len(self.queue) > self.k:
                heapq.heappop(self.queue)
    
    def get_top_k_indexes(self):
        return sorted([(-k, v) for (k, v) in self.queue])

In [4]:
class NoNoise:
    def apply(self, raw, tr):
        return tr

    def apply_batch(self, raw, tr):
        return tr
    
    def __str__(self):
        return "no-noise"
    
    def __repr__(self):
        return "NoNoise()"

In [5]:
from random import Random

class PointSample:
    def __init__(self, p: float, rand: Random):
        """
        Args:
            p: 删掉一个轨迹点的概率
            rand: Random对象
        """
        assert 0 < p <= 1
        self.probability = p
        self.rand = rand
        
    def apply(self, raw, tr):
        tr_len = tr.shape[0]
        indexes = sorted(self.rand.sample(range(tr_len), k=round(tr_len * (1 - self.probability))))
        return tr[indexes]

    def apply_batch(self, raws, trs):
        # trs: (batch_size, tr_len, pt_len)
        tr_len = trs.shape[1]
        indexes = sorted(self.rand.sample(range(tr_len), k=round(tr_len * (1 - self.probability))))
        return trs[:, indexes]

    def __str__(self):
        return f'sample-{self.probability}'
    
    def __repr__(self):
        return f'PointSample(p={self.probability})'

In [6]:
from random import Random


class PointShift:
    def __init__(self, width: int, rand: Random):
        self.width = width
        self.rand = rand
        
    def apply(self, raw, tr):
        tr_len = tr.shape[0]
        width = self.width
        indexes = [i for i in range(tr_len)]
        for i in range(1, tr_len - 1):
            # i下标的点应该移动到to
            this_width = min(width, i, tr_len - i - 1)
            to = i + self.rand.randrange(2 * this_width + 1) - this_width
            if to < i:
                temp = indexes[to]
                for j in range(to, i):
                    indexes[j] = indexes[j + 1]
                indexes[i] = temp
            elif to > i:
                temp = indexes[to]
                for j in reversed(range(i + 1, to + 1)):
                    indexes[j] = indexes[j - 1]
                indexes[i] = temp
        return tr[indexes]
    
    def apply_batch(self, raws, trs):
        tr_len = trs.shape[1]
        width = self.width
        indexes = [i for i in range(tr_len)]
        for i in range(1, tr_len - 1):
            # i下标的点应该移动到to
            this_width = min(width, i, tr_len - i - 1)
            to = i + self.rand.randrange(2 * this_width + 1) - this_width
            if to < i:
                temp = indexes[to]
                for j in range(to, i):
                    indexes[j] = indexes[j + 1]
                indexes[i] = temp
            elif to > i:
                temp = indexes[to]
                for j in reversed(range(i + 1, to + 1)):
                    indexes[j] = indexes[j - 1]
                indexes[i] = temp
        return trs[:, indexes]

    def __str__(self):
        return (f'shift-{self.width}')
    
    def __repr__(self):
        return f'PointShift(width={self.width})'

In [7]:
from random import Random


class SemanticsSubstitution:
    def __init__(self,
                 p: float,
                 keyword_set,
                 raw_to_tr,
                 rand: Random):
        """
        Args:
            p: 替换语义信息的概率
            keyword_set: 可用关键词集合
            raw_to_tr: 原始表示转化为向量表示的函数
            rand: Random对象
        """
        self.probability = p
        self.keyword_set = keyword_set
        self.rand = rand
        self.raw_to_tr = raw_to_tr
    
    def apply(self, raw, tr):
        tr_len = raw.shape[0]
        raw_copy = raw.copy()
        for i in range(tr_len):
            keywords = raw_copy.iloc[i, 4].split(',')
            for j in range(len(keywords)):
                if self.rand.random() < self.probability:
                    keywords[j] = self.rand.choice(self.keyword_set)
            raw_copy.iloc[i, 4] = ','.join(keywords)
        return self.raw_to_tr(raw_copy).to(ctx.device)
    
    def apply_batch(self, raw, tr):
        return torch.stack([self.apply(r, t) for (r, t) in zip(raw, tr)], dim=0)
    
    def __str__(self):
        return (f'substitute-{self.probability}')
    
    def __repr__(self):
        return (f'SemanticsSubstitution(p={self.probability})')

In [8]:
import import_ipynb
from gensim.models import Word2Vec
from functools import partial
from at2vec import get_mat, PretrainModel, BareDataset, TrajectoryDataset, EncoderDecoder
import torch


if __name__ == '__main__':
    # 准备模型与数据
    sp_model = PretrainModel(ctx.num_sp_grids, ctx.sp_len, torch.device('cpu'))
    sp_model.load_state_dict(torch.load(ctx.sp_pretrain_model_path)['model'])
    ts_model = PretrainModel(ctx.num_ts_grids, ctx.ts_len, torch.device('cpu'))
    ts_model.load_state_dict(torch.load(ctx.ts_pretrain_model_path)['model'])
    sm_model = Word2Vec.load(ctx.sm_pretrain_model_path)
    
    model = EncoderDecoder(ctx.sampled_tr_len, ctx.complete_tr_len, ctx.pt_len, ctx.hidden_len,
                           ctx.num_sp_grids, ctx.num_ts_grids, len(sm_model.wv), ctx.device)
    state = torch.load(ctx.at2vec_model_path)
    model.load_state_dict(state['model'])
    
    raw2tr = partial(get_mat, sp_model=sp_model, ts_model=ts_model, sm_model=sm_model)
    
    bare_dataset = BareDataset(None, ctx.test_tr_path, update_ctx=False, ctx=ctx)
    print(f'trajectory count: {len(bare_dataset)}')
    test_dataset = TestDataset(bare_dataset, raw2tr)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=ctx.test_batch_size)

total lines: 21225850
trajectory count: 424517


  0%|          | 0/424517 [00:00<?, ?it/s]

In [9]:
from at2vec import get_mat
from random import Random

# if __name__ == '__main__':
#     noise = NoNoise()
#     evaluator = Evaluator(model=model, noise=noise, k=70)
#     total = len(bare_dataset)
#     _, chosen_tr = test_dataset[34]
#     evaluator.set_chosen_tr_vec(model.get_rep_vector(chosen_tr.to(ctx.device)))
    # i = 0
    # _, raw = bare_dataset[i]
    # _, tr = test_dataset[i]
    # tr1 = PointSample(p=0.125, rand=Random(251)).apply(raw, tr)
    # tr2 = PointShift(4, Random(251)).apply(raw, tr)
    # print(tr, tr1, tr2, sep="\n")

In [ ]:
import math
from tqdm.notebook import tqdm

if __name__ == '__main__':
    ratios = [0.0 for _ in noises]
    
    num_queries = 50
    
    logging.info(f'{num_queries} {len(noises)}')
    for noise in noises:
        logging.info(f'{noise}')
        
    tr_count = len(test_dataset)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=ctx.test_batch_size)
    
    evaluators = [[Evaluator(model=model, noise=noise, k=50) for noise in noises] for _ in range(num_queries)]
    
    for query_id in range(num_queries):
        evaluators_per_query = evaluators[query_id]
        i = random.randrange(tr_count)
        _, chosen_tr = test_dataset[i]        
        chosen_tr_vec = model.get_rep_vector(chosen_tr.to(ctx.device))
        for evaluator in evaluators_per_query:
            evaluator.set_chosen_tr_vec(chosen_tr_vec)
            
    pbar = tqdm(total=math.ceil(tr_count / ctx.test_batch_size))
    pbar1 = tqdm(total=len(noises))
    for i, (indexes, trs) in enumerate(test_dataloader):
        pbar1.reset()
        trs = trs.to(ctx.device)
        raws = []
        for index in indexes:
            # if index.item() % 10 == 0:
            #     print(f'processing trajectory #{index.item()}')
            index = index.item()
            raws.append(test_dataset.bare_dataset[index][1])
        
        for n, noise in enumerate(noises):
            # print(f'batch {i}/{len(test_dataloader)}, noise {noise}')
            applied = model.get_rep_vector(noise.apply_batch(raws, trs))
            for evaluators_per_query in evaluators:
                evaluators_per_query[n].evaluate_batch_applied(indexes, applied)
            pbar1.update(1)
        pbar.update(1)
    pbar.close()
    pbar1.close()

    for query_id in range(num_queries):
        logging.info(f'Query #{query_id}:')
        evaluators_per_query = evaluators[query_id]
        true_results = evaluators_per_query[0].get_top_k_indexes()
        true_tids = [int(tid) for (_, tid) in true_results]
        intersect_cnt = []
        for evaluator in evaluators_per_query:
            logging.info(f'{evaluator.noise}')
            results = evaluator.get_top_k_indexes()
            tids = [int(tid) for (_, tid) in results]
            scores = " ".join([f'{score:.4f}' for (score, _) in results])
            logging.info(" ".join([str(tid) for tid in tids]))
            logging.info(scores)
            intersect_cnt.append(count_intersect(tids, true_tids) / len(tids))
        logging.info(f"intersect: {' '.join([f'{cnt:.4f}' for cnt in intersect_cnt])}")
        for i in range(len(ratios)):
            ratios[i] += intersect_cnt[i]
    
    for i in range(len(noises)):
        logging.info(f'{noises[i]} {ratios[i] / num_queries}')
        print(f'{noises[i]} {ratios[i] / num_queries}')

  0%|          | 0/415 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]